In [3]:
import pandas as pd
import os
# Path to the dataset
fp_path = '../data/raw/wfp_food_prices_som.csv'

# Loading the data (skipping the second row which contains HXL tags)
df_prices = pd.read_csv(fp_path, skiprows=[1])

print(f"Dataset shape: {df_prices.shape}")
df_prices.head()

# keep only date admin1 commodity usdprice
df_prices = df_prices[['date', 'admin1', 'commodity', 'usdprice']]
df_prices.head()

# Extract year, filter for 2000-2024, remove duplicates
df_prices['year'] = df_prices['date'].str[:4].astype(int)
df_prices = df_prices[(df_prices['year'] >= 2000) & (df_prices['year'] <= 2024)]
df_prices = df_prices.drop(columns=['date'])
df_prices = df_prices.drop_duplicates(subset=['year', 'admin1', 'commodity'])
df_prices.head()

# drop the rows with 'nan' values in admin1 column
df_prices = df_prices[df_prices['admin1'].notna()]

# display the 'nan' values in admin1 column
print(len(df_prices[df_prices['admin1'].isna()]))

Dataset shape: (37538, 16)
0


In [4]:
# display unique values of the column admin1_name
print("Unique values in 'admin1':")
print(df_prices['admin1'].unique())
print(len(df_prices['admin1'].unique()))

Unique values in 'admin1':
['Awdal' 'Bakool' 'Banadir' 'Bay' 'Hiraan' 'Juba Dhexe' 'Juba Hoose'
 'Mudug' 'Shabelle Hoose' 'Sool' 'Woqooyi Galbeed' 'Bari' 'Nugaal'
 'Sanaag' 'Togdheer' 'Galgaduud' 'Gedo' 'Shabelle Dhexe']
18


In [5]:
c_path = '../data/raw/somalia-admin1-crop-production.csv'

# Loading the data (skipping the second row which contains HXL tags)
df_crop_production = pd.read_csv(c_path, skiprows=[1])
df_crop_production.head()

# display unique values of the column admin1_name
print("Unique values in 'admin1_name':")
print(df_crop_production['admin1_name'].unique())
# print length of the column admin1_name
print(len(df_crop_production['admin1_name'].unique()))


Unique values in 'admin1_name':
['Middle Juba' 'Lower Juba' 'Gedo' 'Woqooyi Galbeed' 'Togdheer' 'Sool'
 'Sanaag' 'Bari' 'Nugaal' 'Mudug' 'Galgaduud' 'Hiraan' 'Middle Shabelle'
 'Banadir' 'Lower Shabelle' 'Bay' 'Bakool' 'Awdal']
18


In [6]:
import json
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# Load environment variables (API key should be in .env)
load_dotenv()
client = OpenAI(api_key=os.getenv("GOOGLE_API_KEY"),
                base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

# Extract unique values (excluding NaN)
admin1_prices = [x for x in df_prices['admin1'].unique() if str(x) != 'nan']
admin1_crop = [x for x in df_crop_production['admin1_name'].unique() if str(x) != 'nan']

prompt = f"""
I have two datasets with region names for Somalia that use different naming conventions (e.g., Somali vs English).
Match the names from List A to the most equivalent names in List B.

List A (from df_prices['admin1']):
{admin1_prices}

List B (from df_crop_production['admin1_name']):
{admin1_crop}

Return only a JSON object where keys are names from List A and values are the matching names from List B. 
If a name has no match, use null as the value.
"""

response = client.chat.completions.create(
    model="gemini-3-flash-preview",
    messages=[
        {"role": "system", "content": "You are a data engineering assistant specializing in geographic data mapping."},
        {"role": "user", "content": prompt}
    ],
    response_format={ "type": "json_object" }
)

# Parse the mapping
mapping = json.loads(response.choices[0].message.content)

print("Mapping results:")
print(json.dumps(mapping, indent=2))

Mapping results:
{
  "Awdal": "Awdal",
  "Bakool": "Bakool",
  "Banadir": "Banadir",
  "Bay": "Bay",
  "Hiraan": "Hiraan",
  "Juba Dhexe": "Middle Juba",
  "Juba Hoose": "Lower Juba",
  "Mudug": "Mudug",
  "Shabelle Hoose": "Lower Shabelle",
  "Sool": "Sool",
  "Woqooyi Galbeed": "Woqooyi Galbeed",
  "Bari": "Bari",
  "Nugaal": "Nugaal",
  "Sanaag": "Sanaag",
  "Togdheer": "Togdheer",
  "Galgaduud": "Galgaduud",
  "Gedo": "Gedo",
  "Shabelle Dhexe": "Middle Shabelle"
}


In [7]:
df_prices.head()

,admin1,commodity,usdprice,year
788,Awdal,Maize (white),0.22,2000
789,Awdal,Rice (imported),0.45,2000
790,Awdal,Sorghum (red),0.20,2000
791,Bakool,Maize (white),0.78,2000
792,Bakool,Rice (imported),1.89,2000


In [8]:
# Define the indicator mapping to commodities
indicator_mapping = {
    'crop-production.mai.firr.USD': 'Maize (white)',
    'crop-production.mai.noirr.USD': 'Maize (white)',
    'crop-production.ric.noirr.USD': 'Rice (imported)',
    'crop-production.ric.firr.USD': 'Rice (imported)',
    'crop-production.whe.noirr.USD': 'Wheat',
    'crop-production.whe.firr.USD': 'Wheat'
}

# 1. Prepare df_crop_production: map indicators to commodities and sum by region
df_crop_mapped = df_crop_production.copy()
df_crop_mapped['commodity'] = df_crop_mapped['indicator'].map(indicator_mapping)

# Filter for only the commodities we care about
df_crop_mapped = df_crop_mapped.dropna(subset=['commodity'])

# Sum production values by region and commodity
df_crop_summed = df_crop_mapped.groupby(['admin1_name', 'commodity'])['value'].sum().reset_index()
df_crop_summed.rename(columns={'value': 'crop_production_value_usd'}, inplace=True)

# 2. Apply the admin1 mapping to df_prices to create a temporary join key
# We use the 'mapping' dictionary from the previous cell
df_prices['admin1_crop_key'] = df_prices['admin1'].map(mapping)

# 3. Merge the data into df_prices for the year 2018
# First, create a temporary dataframe for 2018 data
df_prices_2018 = df_prices[df_prices['year'] == 2018].copy()

# Merge with the summed crop production data
df_prices_2018 = df_prices_2018.merge(
    df_crop_summed, 
    left_on=['admin1_crop_key', 'commodity'], 
    right_on=['admin1_name', 'commodity'], 
    how='left'
)

# 4. Update the main df_prices with the new column
# Initialize the column with nulls
df_prices['crop_production_value_usd'] = None

# Update only the 2018 rows using the index
df_prices.loc[df_prices['year'] == 2018, 'crop_production_value_usd'] = df_prices_2018['crop_production_value_usd'].values

# Clean up temporary columns
df_prices.drop(columns=['admin1_crop_key'], inplace=True)

print("Crop production data integrated for 2018.")
print(df_prices[df_prices['year'] == 2018].head())

Crop production data integrated for 2018.
      admin1        commodity  usdprice  year crop_production_value_usd
10896  Awdal    Maize (white)     12.52  2018                       0.0
10897  Awdal            Pasta     13.91  2018                       NaN
10898  Awdal  Rice (imported)      9.74  2018                       0.0
10899  Awdal    Sorghum (red)      8.70  2018                       NaN
10900  Awdal  Sorghum (white)      8.70  2018                       NaN


In [9]:
import pandas as pd
import os

security_path = '../data/raw/suite-of-food-security-indicators_som.csv'

# Loading the data (skipping the second row which contains HXL tags)
df_security = pd.read_csv(security_path, skiprows=[1])
df_security.head()

,Iso3,StartDate,EndDate,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag,Note
0,SOM,2000-01-01,2002-12-31,201,'706,Somalia,21010,Average dietary energy supply adequacy (percen...,6121,Value,20002002,2002,%,72,E,NaN
1,SOM,2001-01-01,2003-12-31,201,'706,Somalia,21010,Average dietary energy supply adequacy (percen...,6121,Value,20012003,2003,%,72,E,NaN
2,SOM,2002-01-01,2004-12-31,201,'706,Somalia,21010,Average dietary energy supply adequacy (percen...,6121,Value,20022004,2004,%,72,E,NaN
3,SOM,2003-01-01,2005-12-31,201,'706,Somalia,21010,Average dietary energy supply adequacy (percen...,6121,Value,20032005,2005,%,72,E,NaN
4,SOM,2004-01-01,2006-12-31,201,'706,Somalia,21010,Average dietary energy supply adequacy (percen...,6121,Value,20042006,2006,%,72,E,NaN


In [10]:
# Define the items we want to extract and their new column names
items_to_extract = {
    "Prevalence of undernourishment (percent) (3-year average)": "prevalence_undernourishment_pct",
    "Average dietary energy supply adequacy (percent) (3-year average)": "dietary_energy_adequacy_pct",
    "Percentage of children under 5 years affected by wasting (percent)": "child_wasting_pct"
}

# 1. Filter df_security for the specific items
df_security_filtered = df_security[df_security['Item'].isin(items_to_extract.keys())].copy()

# 2. Clean Year and Value columns
# Ensure Year is integer for matching with df_prices['year']
df_security_filtered['Year'] = pd.to_numeric(df_security_filtered['Year'], errors='coerce')

# Handle numeric values (convert to float, set non-numeric like '<2.5' to NaN)
df_security_filtered['Value'] = pd.to_numeric(df_security_filtered['Value'], errors='coerce')

# 3. Pivot the table so each indicator is a column
df_security_pivoted = df_security_filtered.pivot(index='Year', columns='Item', values='Value').reset_index()

# Rename columns to the requested names
df_security_pivoted.rename(columns=items_to_extract, inplace=True)

# 4. Merge with df_prices on year
df_prices = df_prices.merge(df_security_pivoted, left_on='year', right_on='Year', how='left')

# Drop the redundant Year column from the merge
df_prices.drop(columns=['Year'], inplace=True)

print("Food security indicators integrated into df_prices.")
print(df_prices.dropna(subset=['prevalence_undernourishment_pct', 'dietary_energy_adequacy_pct', 'child_wasting_pct'], how='all').head())

Food security indicators integrated into df_prices.
     admin1          commodity  usdprice  year crop_production_value_usd  \
66   Bakool      Maize (white)      0.78  2002                      None   
67   Bakool    Rice (imported)      2.86  2002                      None   
68   Bakool      Sorghum (red)      0.61  2002                      None   
69  Banadir      Sorghum (red)      0.51  2002                      None   
70  Banadir  Milk (cow, fresh)      1.91  2002                      None   

    dietary_energy_adequacy_pct  child_wasting_pct  \
66                         72.0                NaN   
67                         72.0                NaN   
68                         72.0                NaN   
69                         72.0                NaN   
70                         72.0                NaN   

    prevalence_undernourishment_pct  
66                             70.0  
67                             70.0  
68                             70.0  
69            